In [1]:
using DataFrames
using Plots
using FreqTables
using TextAnalysis

In [2]:
listAtas = readdir("../atas");
corpus = []

for i in 1:length(listAtas)
    f = open("../atas/" * listAtas[i])
    txt = readlines(f)
    push!(corpus,txt)
end

println(length(corpus), " atas" )

228 atas


### char frequency


In [3]:
dict_char = Dict{Char, Int}()
for txt in corpus
    for char in string(txt)
        if haskey(dict_char, char)
         dict_char[char] += 1
      else
         dict_char[char] = 1
      end
    end
end


In [4]:
chars = [];
frequency = [];
for (key, value) in dict_char
    push!(chars,key)
    push!(frequency,value)
end
char_df = DataFrame(Dict("Char"=>chars, "n"=>frequency));
sort!(char_df, [:n, :Char], rev=[true, false])
char_df[1:20,:]

,Char,n
,Any,Any
1,,1206511
2,e,743528
3,a,659885
4,o,614063
5,s,443176
6,r,387184
7,i,371433
8,d,356968
9,n,318473


### word frequency

In [5]:
global aux = string(corpus[1])
for i = 2:length(corpus)
           global aux = aux * " " * string(corpus[i])
           #aux = string(aux, " ",list_por[i])
       end


In [6]:
aux = lowercase(aux)
words = split(replace(aux, r"\P{L}"i => " "))
table = sort(freqtable(words); rev=true)
println(table[1:20])

20-element Named Vector{Int64}
Dim1       │ 
───────────┼──────
"de"       │ 93952
"a"        │ 41197
"em"       │ 39293
"e"        │ 32085
"o"        │ 27824
"do"       │ 24770
"da"       │ 21635
"no"       │ 18766
"para"     │ 17303
"que"      │ 14345
"com"      │ 11471
"se"       │ 11214
"os"       │  9845
"as"       │  9599
"dos"      │  9403
"na"       │  8355
"preços"   │  7942
"ao"       │  7687
"inflação" │  7643
"das"      │  6999


In [7]:
Mystopwords = ["de",  "a",  "o",  "que",  "e",  "do",  "da",  "em",  "um",  "para",  "com",  "não",  "uma",  "os",  "no",  "se",  "na", 
    "por",  "mais",  "as",  "dos",  "como",  "mas",  "ao",  "ele",  "das",  "à",  "seu",  "sua",  "ou",  "quando",  "muito",  "nos",  "já",  
    "eu",  "também",  "só",  "pelo",  "pela",  "até",  "isso",  "ela",  "entre",  "depois",  "sem",  "mesmo",  "aos",  "seus",  "quem", 
    "nas",  "me",  "esse",  "eles",  "você",  "essa",  "num",  "nem",  "suas",  "meu",  "às",  "minha",  "numa",  "pelos",  "elas",  "qual", 
    "nós",  "lhe",  "deles",  "essas",  "esses",  "pelas",  "este",  "dele",  "tu",  "te",  "vocês",  "vos",  "lhes",  "meus",  "minhas", 
    "teu",  "tua",  "teus",  "tuas",  "nosso",  "nossa",  "nossos",  "nossas",  "dela",  "delas",  "esta",  "estes",  "estas",  "aquele", 
    "aquela",  "aqueles",  "aquelas",  "isto",  "aquilo",  "estou",  "está",  "estamos",  "estão",  "estive",  "esteve",  "estivemos", 
    "estiveram",  "estava",  "estávamos",  "estavam",  "estivera",  "estivéramos",  "esteja",  "estejamos",  "estejam",  "estivesse", 
    "estivéssemos",  "estivessem",  "estiver",  "estivermos",  "estiverem",  "hei",  "há",  "havemos",  "hão",  "houve",  "houvemos",  
    "houveram",  "houvera",  "houvéramos",  "haja",  "hajamos",  "hajam",  "houvesse",  "houvéssemos",  "houvessem",  "houver", 
    "houvermos",  "houverem",  "houverei",  "houverá",  "houveremos",  "houverão",  "houveria",  "houveríamos",  "houveriam",  "sou",  
    "somos",  "são",  "era",  "éramos",  "eram",  "fui",  "foi",  "fomos",  "foram",  "fora",  "fôramos",  "seja",  "sejamos",  "sejam", 
    "fosse",  "fôssemos",  "fossem",  "for",  "formos",  "forem",  "serei",  "será",  "seremos",  "serão",  "seria",  "seríamos",  "seriam", 
    "tenho",  "tem",  "temos",  "tém",  "tinha",  "tínhamos",  "tinham",  "tive",  "teve",  "tivemos",  "tiveram",  "tivera",  "tivéramos", 
    "tenha",  "tenhamos",  "tenham",  "tivesse",  "tivéssemos",  "tivessem",  "tiver",  "tivermos",  "tiverem",  "terei",  "terá",  "teremos",
    "terão",  "teria",  "teríamos",  "teriam", "sobre","então","onde","pra","vamos","lá","sim","não","bem","tudo","agora","aqui","é",
    "acordar","agora","ainda","aladi","alegrar","além","antar","ante","anthero","antonio","apenas","apesar","apresentação","aquém","araújo","cada","capitar","carioca","carteiro","contra","corpus","corrêa","costa","daquela","demais","diante","edson","entanto","estar","estevar","então","feltrim","final","finar","geral","içar","ie","intuito",
"le","luiz","luzir","mediante","meirelles","mercar","moraes","necessariamente","neto","of","oficiar","oliveira","onde","ora","parir","paulo","pelar","pesar","pilar","pois","primo","quadrar","reinar","res","resinar","reunião","ser","sob","sobre","somente","sr","tal","tais","tanto","thomson","tipo","todo","tony","usecheque","vasconcelos",
"a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z",
"um","dois","três","quatro","cinco","seis","sete","oito","nove","dez","onze","doze","treze","catorze","quinze","dezesseis","dezessete","dezoito","dezenove","vinte",
"aquela","aquelas","aquele","aqueles","àquela","àquelas","daquele","daqueles","daquela","daquelas","naquele","naqueles","naquela","naquelas","neste","nesta","nestes","nestas","nisto","nesse","nessa","nesses","nessas","nisso",
"janeiro","fevereiro","março","abril","maio","junho","julho","agosto","setembro","outubro","novembro","dezembro","mês","meses","ano","anos"]

println("Number of stopwords: ",length(Mystopwords))

Number of stopwords: 380


In [8]:
words_without_stopwords = []
for w in words
    if ~(w in Mystopwords)
        push!(words_without_stopwords, w)
    end
end
println(length(words))
println(length(words_without_stopwords))

1135848
579085


In [9]:
table2 = sort(freqtable(words_without_stopwords); rev=true)
println(table2[1:20])

20-element Named Vector{Int64}
Dim1          │ 
──────────────┼─────
"preços"      │ 7942
"inflação"    │ 7643
"taxa"        │ 4563
"relação"     │ 4330
"copom"       │ 4001
"bilhões"     │ 3949
"crescimento" │ 3696
"produção"    │ 3349
"bens"        │ 3326
"período"     │ 3127
"anterior"    │ 3109
"cenário"     │ 3101
"política"    │ 3040
"us"          │ 2979
"monetária"   │ 2933
"índice"      │ 2680
"expansão"    │ 2523
"mercado"     │ 2490
"economia"    │ 2378
"aumento"     │ 2367


### bigram

In [10]:
dict_bigram = Dict{Any, Any}()
for sent in corpus
    doc = StringDocument(lowercase(string(sent)))
    dict = ngrams(doc, 2) #this function includes 1-gram and 2-grams
    for k in dict
        if ' ' in k[1] # exclude 1-gram
            if haskey(dict_bigram, k[1])
                dict_bigram[k[1]] += k[2]
            else
                dict_bigram[k[1]] = k[2]
            end
        end
    end
end

In [11]:
bigrams = [];
frequency = [];
for (key, value) in dict_bigram
    push!(bigrams,key)
    push!(frequency,value)
end
bigram_df = DataFrame(Dict("Bigram"=>bigrams, "n"=>frequency));
sort!(bigram_df, [:n, :Bigram], rev=[true, false])
bigram_df[1:20,:]

,Bigram,n
,Any,Any
1,""" ,",28557
2,", """,28529
3,""" """,14735
4,% em,10866
5,", o",6829
6,", a",6111
7,"% ,",6064
8,\\ $,5633
9,", em",4993


### trigram

In [12]:
dict_trigram = Dict{Any, Any}()
for sent in corpus
    doc = StringDocument(lowercase(string(sent)))
    dict = ngrams(doc, 3) #this function includes 1-gram, 2-grams and 3-grams
    for k in dict
        if count( c-> (c == ' ') , k[1]) == 2 # exclude 1-gram and 2-gram
            if haskey(dict_trigram, k[1])
                dict_trigram[k[1]] += k[2]
            else
                dict_trigram[k[1]] = k[2]
            end
        end
    end
end

In [13]:
trigrams = [];
frequency = [];
for (key, value) in dict_trigram
    push!(trigrams,key)
    push!(frequency,value)
end
trigram_df = DataFrame(Dict("Trigram"=>trigrams, "n"=>frequency));
sort!(trigram_df, [:n, :Trigram], rev=[true, false])
trigram_df[1:20,:]

,Trigram,n
,Any,Any
1,""" , """,28529
2,", "" """,14735
3,""" "" ,",14732
4,us \\ $,2818
5,r \\ $,2644
6,". "" ,",2253
7,de bens de,1645
8,em relação ao,1596
9,em doze meses,1569
